In [130]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd


In [131]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [132]:
# Define database and collection
db = client.craigslist_houses_db
collection = db.items

In [133]:
# URL of page to be scraped
# craigslist homes for sale in McLean
url = 'https://washingtondc.craigslist.org/search/rea?query=mclean&availabilityMode=0&sale_date=all+dates'

In [134]:
# Retrieve page with the requests module
response = requests.get(url)

In [135]:
# Create BeautifulSoup object; parse with 'html.parser'
#soup = BeautifulSoup(response.text, 'html.parser')
#or
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [136]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   washington, DC real estate "mclean" - craigslist
  </title>
  <meta content='washington, DC real estate "mclean" - craigslist' name="description"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <link href="https://washingtondc.craigslist.org/search/rea" rel="canonical"/>
  <link href="https://washingtondc.craigslist.org/search/rea?availabilityMode=0&amp;format=rss&amp;query=mclean" rel="alternate" title='RSS feed for craigslist | washington, DC real estate "mclean" - craigslist' type="application/rss+xml"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <link href="//www.craigslist.org/styles/cl.css?v=93aa8b9183682fd14ca12ff38c0d801c" media="all" rel="stylesheet" type="text/css"/>
  <link href="//www.craigslist.org/styles/search.css?v=84cf86bc094026e12fa066bbbab154ac" media="all" rel="stylesheet" type="text/css"/>
  <link href="//www.craigslist.org/styles/jquery-ui-clcustom.css?v=3b05ddffb7

In [137]:
# results are returned as an iterable list
results = soup.find_all('li', class_="result-row")

In [138]:
def url_to_address(url):
    try:             
        response = requests.get(url)
        soup2 = BeautifulSoup(response.text, 'lxml')
        results2 = soup2.find('div', class_="mapaddress").text
      
    except: 
        results2 = "no address"
            
            

    return results2
    

In [139]:
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        # Identify and return price of listing
        price = result.a.span.text
        # Identify and return link to listing
        link = result.a['href']
        # identify the address
        address = url_to_address(link)
        
        # Print results only if title, price, and link are available
        if (title and price and link):
            print('-------------')
            print(title)
            print(price)
            print(link)
            print(address)
            
        #go to link, use request library to grab, make a new soup2, extract address from new page
        
        address = url_to_address(link)
        
             
          # Dictionary to be inserted as a MongoDB document
        post = {
                'title': title,
                'price': price,
                'url': link,
                'address': address
                   }

        collection.insert_one(post)
            
    except AttributeError as e:
        print(e)

-------------
RARE VACANCY in this popular McLean Neighborhood
$4500
https://washingtondc.craigslist.org/nva/reb/d/mc-lean-rare-vacancy-in-this-popular/6827441885.html
9014 Old Dominion Drive
-------------
Great Colonial House in McLean, VA -- 18 Min. to D.C. or Crystal City.
$1250000
https://washingtondc.craigslist.org/nva/reo/d/mc-lean-great-colonial-house-in-mclean/6825971559.html
6533 Sunny Hill Court
-------------
Great Colonial House in McLean, VA -- 18 Min. to D.C. or Crystal City.
$1250000
https://washingtondc.craigslist.org/nva/reo/d/mc-lean-great-colonial-house-in-mclean/6825971842.html
6533 Sunny Hill Court
-------------
Contemporary Detached 4 br 2.5 ba in McLean, VA
$4150
https://washingtondc.craigslist.org/nva/reb/d/mc-lean-contemporary-detached-4-br-25/6825752844.html
Woodsong Ct. at Twisting Tree Ln.
-------------
7 Bedroom 7 Bathroom (2HB) House For Sale in McLean
$2350000
https://washingtondc.craigslist.org/nva/reb/d/mc-lean-7-bedroom-7-bathroom-2hb-house/6808290394.h

In [140]:
#url = 'https://washingtondc.craigslist.org/nva/reo/d/mc-lean-great-colonial-house-in-mclean/6825971559.html'
#response = requests.get(url)
#soup2 = BeautifulSoup(response.text, 'lxml')
#results2 = soup2.find('div', class_="mapaddress").text
#print(soup2.prettify())
#results2

In [141]:
#url = 'https://washingtondc.craigslist.org/nva/reo/d/mc-lean-great-colonial-house-in-mclean/6825971559.html'
#url_to_address(url)

In [172]:
#!pip install python-zillow
collection.count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


34

In [143]:
import zillow

#with open("./bin/config/zillow_key.conf", 'r') as f:
 #   key = f.readline().replace("\n", "")



api = zillow.ValuationApi()

In [144]:
import pprint

In [145]:
#address = "6533 Sunny Hill Court"


In [146]:
#data = api.GetSearchResults(key, address, postal_code)

ZillowError: {'message': 'Zillow did not return a valid response: <?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/091b98a/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address>Lincoln Way at Jones Branch Drive</address><citystatezip>22102</citystatezip></request><message><text>Error: no exact match found for input address</text><code>508</code></message></SearchResults:searchresults><!-- H:026  T:32ms  S:150  R:Mon Feb 25 13:11:14 PST 2019  B:5.0.58770.2-hotfix_2019-02-20.02147e7~hotfix_adt_1053.88b3887 -->'}

In [ ]:
#data.zestimate.amount

In [147]:
db = client.zillow_market_db
collection2 = db.items

#collection.count()

In [150]:
postal_code = "22102"
key = "X1-ZWz1gx8me6fbbf_a57uf"

for record in collection.find():
    try:
        
        #print(record["address"])
        list_address = record["address"]
        data = api.GetSearchResults(key, list_address, postal_code)

               # Dictionary to be inserted as a MongoDB document
        marketvalue = {
                    'address': list_address,
                    'zillow_price': data.zestimate.amount,
                          }

        collection2.insert_one(marketvalue)
            
    except:
        marketvalue = {
                'address': list_address,
                'zillow_price': -1,
                          }

        collection2.insert_one(marketvalue)
    
    

In [153]:
for record in collection2.find():
    print(record)

{'_id': ObjectId('5c745a8c737ae12260fec0c3'), 'address': '9014 Old Dominion Drive', 'zillow_price': 1283701}
{'_id': ObjectId('5c745a8c737ae12260fec0c4'), 'address': '6533 Sunny Hill Court', 'zillow_price': 1272049}
{'_id': ObjectId('5c745a8c737ae12260fec0c5'), 'address': '6533 Sunny Hill Court', 'zillow_price': 1272049}
{'_id': ObjectId('5c745b5b737ae12260fec0c6'), 'address': '9014 Old Dominion Drive', 'zillow_price': 1283701}
{'_id': ObjectId('5c745b5c737ae12260fec0c7'), 'address': '6533 Sunny Hill Court', 'zillow_price': 1272049}
{'_id': ObjectId('5c745b5c737ae12260fec0c8'), 'address': '6533 Sunny Hill Court', 'zillow_price': 1272049}
{'_id': ObjectId('5c745b5c737ae12260fec0c9'), 'address': 'Woodsong Ct. at Twisting Tree Ln.', 'zillow_price': -1}
{'_id': ObjectId('5c745b5c737ae12260fec0ca'), 'address': 'no address', 'zillow_price': -1}
{'_id': ObjectId('5c745b5d737ae12260fec0cb'), 'address': 'James Dr at Fardale St', 'zillow_price': -1}
{'_id': ObjectId('5c745b5d737ae12260fec0cc'), 

In [154]:
cursor = collection2.find()
# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))
df

,_id,address,zillow_price
0,5c745a8c737ae12260fec0c3,9014 Old Dominion Drive,1283701
1,5c745a8c737ae12260fec0c4,6533 Sunny Hill Court,1272049
2,5c745a8c737ae12260fec0c5,6533 Sunny Hill Court,1272049
3,5c745b5b737ae12260fec0c6,9014 Old Dominion Drive,1283701
4,5c745b5c737ae12260fec0c7,6533 Sunny Hill Court,1272049
5,5c745b5c737ae12260fec0c8,6533 Sunny Hill Court,1272049
6,5c745b5c737ae12260fec0c9,Woodsong Ct. at Twisting Tree Ln.,-1
7,5c745b5c737ae12260fec0ca,no address,-1
8,5c745b5d737ae12260fec0cb,James Dr at Fardale St,-1
9,5c745b5d737ae12260fec0cc,Cherri Dr at Leonard Rd,-1


In [155]:
#collection.delete_many({})

In [156]:
#collection.count()

In [158]:
df.head()

,_id,address,zillow_price
0,5c745a8c737ae12260fec0c3,9014 Old Dominion Drive,1283701
1,5c745a8c737ae12260fec0c4,6533 Sunny Hill Court,1272049
2,5c745a8c737ae12260fec0c5,6533 Sunny Hill Court,1272049
3,5c745b5b737ae12260fec0c6,9014 Old Dominion Drive,1283701
4,5c745b5c737ae12260fec0c7,6533 Sunny Hill Court,1272049


In [171]:
df1 = df.groupby("address")["zillow_price"].mean()
df1 = pd.DataFrame(df1)
df1 = df1.reset_index()
df1.sort_values(by = "zillow_price", ascending=False)


,address,zillow_price
4,9014 Old Dominion Drive,1283701
1,6533 Sunny Hill Court,1272049
0,"121 Martin Lane, Alexandria, VA 22304",625655
2,"6805 Coach Way, Haymarket, VA 20169",-1
3,8220 Crestwood Heights Drive,-1
5,Browns Mill Road at Rosewood Hill Drive,-1
6,Cherri Dr at Leonard Rd,-1
7,James Dr at Fardale St,-1
8,Lincoln Way at Jones Branch Drive,-1
9,Woodsong Ct. at Twisting Tree Ln.,-1


In [185]:
cursor = collection.find()
# Expand the cursor and construct the DataFrame
df_cl =  pd.DataFrame(list(cursor))
df_cl.head()
df_cl = df_cl.groupby("address").first()
df_cl

,_id,price,title,url
address,,,,
"121 Martin Lane, Alexandria, VA 22304",5c7459eb737ae12260fec0ac,$610000,Stunning 3BR 3.5BA 4 Lvl Luxury Townhome in Ca...,https://washingtondc.craigslist.org/nva/reb/d/...
6533 Sunny Hill Court,5c7459e7737ae12260fec0a2,$1250000,"Great Colonial House in McLean, VA -- 18 Min. ...",https://washingtondc.craigslist.org/nva/reo/d/...
"6805 Coach Way, Haymarket, VA 20169",5c7459eb737ae12260fec0ae,$459000,Price just reduced! Stunning home in great Hay...,https://washingtondc.craigslist.org/nva/reb/d/...
8220 Crestwood Heights Drive,5c7459ec737ae12260fec0b0,$988888,OPEN HOUSE SUNDAY - FEBRUARY 24TH FROM 2 PM TO...,https://washingtondc.craigslist.org/nva/reb/d/...
9014 Old Dominion Drive,5c7459e7737ae12260fec0a1,$4500,RARE VACANCY in this popular McLean Neighborhood,https://washingtondc.craigslist.org/nva/reb/d/...
Browns Mill Road at Rosewood Hill Drive,5c7459ec737ae12260fec0b1,$747900,EXCELLENT LOCATION AND PRICE,https://washingtondc.craigslist.org/nva/reo/d/...
Cherri Dr at Leonard Rd,5c7459e9737ae12260fec0a7,$500000,Attractive Price lot to Build Your Dream Home,https://washingtondc.craigslist.org/nva/reb/d/...
James Dr at Fardale St,5c7459e9737ae12260fec0a6,$680000,Great Potential - Renovate/Expand/Build,https://washingtondc.craigslist.org/nva/reb/d/...
Lincoln Way at Jones Branch Drive,5c7459f2737ae12260fec0c2,$385000,Luxury Two Bedroom Two Bath Condo - Perfect Lo...,https://washingtondc.craigslist.org/nva/reo/d/...


In [202]:
df_merge = pd.merge(df1, df_cl, how="left", on="address")
df_merge.sort_values("zillow_price", ascending=False)
#make sure price is not an integar
df_merge["price"] = df_merge["price"].apply(lambda x: int(x.replace("$","")))
df_merge.head()
df_merge["price difference"] = df_merge["zillow_price"] - df_merge["price"]
df_merge.sort_values("price difference", ascending=False)

,address,zillow_price,_id,price,title,url,price difference
4,9014 Old Dominion Drive,1283701,5c7459e7737ae12260fec0a1,4500,RARE VACANCY in this popular McLean Neighborhood,https://washingtondc.craigslist.org/nva/reb/d/...,1279201
1,6533 Sunny Hill Court,1272049,5c7459e7737ae12260fec0a2,1250000,"Great Colonial House in McLean, VA -- 18 Min. ...",https://washingtondc.craigslist.org/nva/reo/d/...,22049
0,"121 Martin Lane, Alexandria, VA 22304",625655,5c7459eb737ae12260fec0ac,610000,Stunning 3BR 3.5BA 4 Lvl Luxury Townhome in Ca...,https://washingtondc.craigslist.org/nva/reb/d/...,15655
9,Woodsong Ct. at Twisting Tree Ln.,-1,5c7459e8737ae12260fec0a4,4150,"Contemporary Detached 4 br 2.5 ba in McLean, VA",https://washingtondc.craigslist.org/nva/reb/d/...,-4151
8,Lincoln Way at Jones Branch Drive,-1,5c7459f2737ae12260fec0c2,385000,Luxury Two Bedroom Two Bath Condo - Perfect Lo...,https://washingtondc.craigslist.org/nva/reo/d/...,-385001
2,"6805 Coach Way, Haymarket, VA 20169",-1,5c7459eb737ae12260fec0ae,459000,Price just reduced! Stunning home in great Hay...,https://washingtondc.craigslist.org/nva/reb/d/...,-459001
6,Cherri Dr at Leonard Rd,-1,5c7459e9737ae12260fec0a7,500000,Attractive Price lot to Build Your Dream Home,https://washingtondc.craigslist.org/nva/reb/d/...,-500001
7,James Dr at Fardale St,-1,5c7459e9737ae12260fec0a6,680000,Great Potential - Renovate/Expand/Build,https://washingtondc.craigslist.org/nva/reb/d/...,-680001
5,Browns Mill Road at Rosewood Hill Drive,-1,5c7459ec737ae12260fec0b1,747900,EXCELLENT LOCATION AND PRICE,https://washingtondc.craigslist.org/nva/reo/d/...,-747901
3,8220 Crestwood Heights Drive,-1,5c7459ec737ae12260fec0b0,988888,OPEN HOUSE SUNDAY - FEBRUARY 24TH FROM 2 PM TO...,https://washingtondc.craigslist.org/nva/reb/d/...,-988889
